# Getting and cleaning data

This database shows the number of students enrolled in Dutch higher education institutions, for the years 2010 to 2020

Three questions I am interested in:

- (How) did the transfer to a new funding model change the subject people decided to study?
- (How) do changing real estate prices change the city people decide to study in?
- (How) do gender ratios change over time in different areas of study (e.g. stem, humanities)?

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data_highered_raw.csv", sep = ";", low_memory=False)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

# Data cleaning: substituting meaningful names for coded variables 

The first big task here is to substitute all values with meaningful values. I have another csv with metadata.

In [3]:
metadata = pd.read_csv("metadata_highered.csv", sep = ";")

In [ ]:
metadata.head(60)

For my purposes, I will only need the first two columns - ID (which contains the code) and Title (which contains the human title). So I'll extract those:

In [4]:
metadata = metadata[['ID', 'Title']].copy()

And I can drop the first ten rows, which contain info I do not need right now.

In [5]:
metadata = metadata.iloc[11:].copy()

I can also drop all the rows that say 'Key' (they are sort of sub-column headers)

In [6]:
metadata = metadata[metadata['ID'] != 'Key'].copy()

In [7]:
# Then I'll gather the column names for each of the rows that need decoding 

to_decode = [row for row in metadata['ID'] if row in df.columns]        

to_decode

['Geslacht',
 'Migratieachtergrond',
 'Onderwijssoort',
 'StudierichtingISCED2013',
 'Perioden']

In [8]:
indexes = []
for item in to_decode: 
    indexes.append(metadata[metadata['ID'] == item].index)

Next I'll get the values *between* the indexes of the column names. (This process could undoubtedly be more streamlined and easier to read, but oh well - here it is)

In [9]:
Geslacht = metadata.loc[indexes[0][0]+1:indexes[1][0]-1]

In [10]:
Migratieachtergrond = metadata.loc[indexes[1][0]+1:indexes[2][0]-1]

In [11]:
Onderwijssoort = metadata.loc[indexes[2][0]+1:indexes[3][0]-1]

In [12]:
StudierichtingISCED2013 = metadata.loc[indexes[3][0]+1:indexes[4][0]-1]

In [13]:
Perioden = metadata.loc[indexes[4][0]+1:]

Then I'll use these in a function.

In [14]:
# OK this function took me FOREVER to get right, but BOY does it work. 


def decodevalues(column, dataframe):
    
    newvalues = []
    
    for item in df[column]: 
        for i in range(len(dataframe)):        
            if item == dataframe['ID'].iloc[i]:
                newvalues.append(dataframe['Title'].iloc[i])

    df[column]=newvalues
    return

In [15]:
decodevalues("Geslacht", Geslacht)

In [16]:
decodevalues("Migratieachtergrond", Migratieachtergrond)

In [17]:
decodevalues("Onderwijssoort", Onderwijssoort)

In [18]:
decodevalues("StudierichtingISCED2013", StudierichtingISCED2013)

In [19]:
#For 'perioden' it's actually better to follow a different route and just strip off the bit at the end.
#I'll keep in mind that here '2010' means 'The academic year that starts in summer 2010', so 2010/2011.

In [20]:
def replacer(x):
    return x.replace('SJ00','')

In [21]:
df['Perioden'] = df['Perioden'].apply(replacer)

### Next I'll rename some of the columns (and drop column 'ID', which is adequately represented in the dataframe's ID)

In [22]:
df = df.drop(["ID"], axis = 1).copy()

In [23]:
df = df.rename(columns = {"StudierichtingISCED2013": "Studierichting", "TotaalIngeschrevenen_1": "Enrollment", "Eerstejaarsstudenten_2": "Freshmen", "Ouderejaarsstudenten_3": "Non_freshmen"}).copy()

In [24]:
df.reset_index(inplace = True, drop = True)

In [25]:
df.head(60)

,Geslacht,Migratieachtergrond,Onderwijssoort,Studierichting,Perioden,Enrollment,Freshmen,Non_freshmen
0,Totaal mannen en vrouwen,Totaal migratieachtergrond,Hoger onderwijs,Totaal,2010,656706,134101,522605
1,Totaal mannen en vrouwen,Totaal migratieachtergrond,Hoger onderwijs,Totaal,2011,667194,135114,532080
2,Totaal mannen en vrouwen,Totaal migratieachtergrond,Hoger onderwijs,Totaal,2012,661122,133699,527423
3,Totaal mannen en vrouwen,Totaal migratieachtergrond,Hoger onderwijs,Totaal,2013,688191,143387,544804
4,Totaal mannen en vrouwen,Totaal migratieachtergrond,Hoger onderwijs,Totaal,2014,699572,138638,560934
5,Totaal mannen en vrouwen,Totaal migratieachtergrond,Hoger onderwijs,Totaal,2015,701118,132066,569052
6,Totaal mannen en vrouwen,Totaal migratieachtergrond,Hoger onderwijs,Totaal,2016,712113,141325,570788
7,Totaal mannen en vrouwen,Totaal migratieachtergrond,Hoger onderwijs,Totaal,2017,730146,151011,579135
8,Totaal mannen en vrouwen,Totaal migratieachtergrond,Hoger onderwijs,Totaal,2018,747841,156778,591063
9,Totaal mannen en vrouwen,Totaal migratieachtergrond,Hoger onderwijs,Totaal,2019,767577,159306,608271


I'll also drop "Non_freshmen", since I am really only interested in the freshmen (and I'll keep "Enrollment" around so that I can re-calculate the number of non-freshmen)

In [26]:
df = df.drop(columns = ["Non_freshmen"]).copy()

In [27]:
df.shape

(186516, 7)

#### I am also going to look only at the aggregated data on migration background. So I'm keeping only rows that have Migratieachtergrond == Totaal Migratieachtergrond

In [28]:
df = df[df["Migratieachtergrond"]=="Totaal migratieachtergrond"].copy()

In [29]:
df.shape

(16956, 7)

Then I can also drop the whole column Migratieachtergrond, since there's only a single value left in it. 

In [30]:
df = df.drop(columns = ['Migratieachtergrond']).dopy()

I also figured out that within 'onderwijssoort', 'hoger onderwijs' (higher education) is the total of 'hoger beroepsonderwijs' (professional degrees) and 'wetenschappelijk onderwijs' (university degrees). Since I want to look at these two separately, I will get rid of the 'Hoger Onderwijs' values.

The reason I want to look at them separately is that I want to see if the change in the funding model had a big impact on the choices people made - and I would expect this to be more visible in 'wetenschappelijk onderwijs'. Also, I would not be surprised if the ratio between professional and university degrees changed.

In [32]:
df = df[df["Onderwijssoort"]!="Hoger onderwijs"].copy()

In [40]:
df["Geslacht"].value_counts()

Totaal mannen en vrouwen    3768
Mannen                      3768
Vrouwen                     3768
Name: Geslacht, dtype: int64

Finally, it would be good to have one dataframe that does not include gender, and one that does. I'll try out different experiments with both of these later. 

In [47]:
df_total = df[df["Geslacht"]=="Totaal mannen en vrouwen"]

In [48]:
df_gendered = df[df["Geslacht"]!="Totaal mannen en vrouwen"]

In [50]:
df_gendered.shape

(7536, 6)

In [51]:
df_total.shape

(3768, 6)

I can drop the gender column from the df_total dataframe too.

In [52]:
df_total = df_total.drop(columns = ["Geslacht"])

In [53]:
df_total.to_csv("data_neutral.csv", sep = ";")

In [54]:
df_gendered.to_csv("data_gendered.csv", sep = ";")